In [1]:
# This notebook is only for preprocessing the data which contains:
    # 1. Importing the dataframes sourced from SQL
    # 2. Pivotting the LABEL table's values with label_value and maks_probability columns
    # 3. Merging the clip_encode data with pivot table created on the previous step
    # 4. Dropping unwanted columns such as ID columns & VMAF
    # 5. Train and Test data split with 20%
    # 6. Converting categorical columns to numericals
    # 7. Exporting Train and Test datasets
    # /// please use exported datasets to build models ///

In [2]:
# Import Libraries
import pandas as pd
from sklearn import preprocessing
from joblib import dump, load
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
import numpy as np


In [3]:
# Load clip & encode dataset
df_clip_encode = pd.read_csv("../clip_encode_merged_data.csv", encoding = 'utf8')
df_clip_encode.head(5)

,encode_id,clip_id,encode_width,encode_height,clip_width,clip_height,clip_duration,clip_size,clip_bitrate_total,video_profile,clip_frame_rate,crf,encode_bitrate_video,nr_of_images,nr_of_image_shifts,psnr,vmaf
0,7,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,18,18667784.0,3,8,44.621550,99.406406
1,8,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,19,16379868.0,3,8,44.051502,99.192502
2,9,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,20,14293883.0,3,8,43.408926,98.924590
3,10,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,27,3985062.0,3,8,38.802828,89.462670
4,11,5,1920,1080,1920,1080,19,41249088,17350686,High 4:2:2,30000/1001,29,2969404.0,3,8,37.790129,84.728349


In [4]:
# Load Label dataset
df_label = pd.read_csv("../label_data.csv", encoding = 'utf8')
df_label.head(5)

,label_clip_id,label_value,maks_probability
0,5,"chime, bell, gong",57.84
1,5,"spider web, spider's web",68.54
2,5,"flagpole, flagstaff",60.99
3,8,"mobile home, manufactured home",56.77
4,6,"African chameleon, Chamaeleo chamaeleon",81.73


In [5]:
# Load Scene Change dataset
df_scene = pd.read_csv("../scene_change.csv", encoding = 'utf8')
df_scene.head(5)

,scene_clip_id,scene_percentage,nr_of_scene_changes
0,6,probability_0.1,11
1,6,probability_0.2,9
2,6,probability_0.3,9
3,6,probability_0.4,9
4,6,probability_0.5,5


## Pivotting Label dataset

In [6]:
df_label_pivot = pd.pivot_table(df_label, values='maks_probability', index=['label_clip_id'],
                     columns=['label_value'], aggfunc=np.sum, fill_value = 0)
df_label_pivot.head(5)

label_value,"African chameleon, Chamaeleo chamaeleon","American egret, great white heron, Egretta albus",Dutch oven,abaya,alp,altar,analog clock,"balance beam, beam",balloon,"ballplayer, baseball player",...,trench coat,turnstile,"vacuum, vacuum cleaner","wardrobe, closet, press","web site, website, internet site, site",wig,window screen,window shade,wine bottle,"worm fence, snake fence, snake-rail fence, Virginia fence"
label_clip_id,,,,,,,,,,,,,,,,,,,,,
5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,81.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,56.98,0.0,0.0,0.0
8,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,73.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


## Pivotting Scene Change dataset

In [7]:
df_scene_pivot = pd.pivot_table(df_scene, values='nr_of_scene_changes', index=['scene_clip_id'],
                     columns=['scene_percentage'], aggfunc=np.sum, fill_value = 0)
df_scene_pivot.head(5)

scene_percentage,probability_0.1,probability_0.2,probability_0.3,probability_0.4,probability_0.5,probability_0.6,probability_0.7,probability_0.8,probability_0.9
scene_clip_id,,,,,,,,,
6,11,9,9,9,5,3,0,0,0
7,1826,114,88,56,24,6,2,0,0
8,1,1,1,1,1,1,1,0,0
9,11,9,9,9,5,3,0,0,0
11,11,9,9,9,5,3,0,0,0


## Merge datasets: clip & encode & scene with label on clip_id

In [8]:
df_merged = df_clip_encode.merge(df_label_pivot, how = 'inner', left_on='clip_id', right_on='label_clip_id')
df_merged = df_merged.merge(df_scene_pivot, how = 'inner', left_on = 'clip_id', right_on = 'scene_clip_id')
df_merged.head()

,encode_id,clip_id,encode_width,encode_height,clip_width,clip_height,clip_duration,clip_size,clip_bitrate_total,video_profile,...,"worm fence, snake fence, snake-rail fence, Virginia fence",probability_0.1,probability_0.2,probability_0.3,probability_0.4,probability_0.5,probability_0.6,probability_0.7,probability_0.8,probability_0.9
0,140,6,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,...,0.0,11,9,9,9,5,3,0,0,0
1,141,6,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,...,0.0,11,9,9,9,5,3,0,0,0
2,142,6,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,...,0.0,11,9,9,9,5,3,0,0,0
3,143,6,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,...,0.0,11,9,9,9,5,3,0,0,0
4,144,6,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,...,0.0,11,9,9,9,5,3,0,0,0


In [9]:
# drop unwanted columns
df_merged = df_merged.drop(columns=['encode_id', 'clip_id', 'vmaf'], axis=1)
#df_merged = df_merged.drop(columns=['encode_id', 'clip_id', 'psnr', 'nr_of_images', 'nr_of_image_shifts'], axis=1)

In [10]:
#drop NULL rows
#here I simply dropped the null values and ofcourse lost some information. Later I will try to ampute data with mean.
print(df_merged.shape)
df_merged = df_merged.dropna()
print(df_merged.shape)

(5234, 160)
(2169, 160)


In [11]:
df_merged.head(5)

,encode_width,encode_height,clip_width,clip_height,clip_duration,clip_size,clip_bitrate_total,video_profile,clip_frame_rate,crf,...,"worm fence, snake fence, snake-rail fence, Virginia fence",probability_0.1,probability_0.2,probability_0.3,probability_0.4,probability_0.5,probability_0.6,probability_0.7,probability_0.8,probability_0.9
0,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,30000/1001,18,...,0.0,11,9,9,9,5,3,0,0,0
1,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,30000/1001,19,...,0.0,11,9,9,9,5,3,0,0,0
2,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,30000/1001,20,...,0.0,11,9,9,9,5,3,0,0,0
3,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,30000/1001,23,...,0.0,11,9,9,9,5,3,0,0,0
4,1920,1080,1920,1080,19,378608416,159254815,High 4:2:2,30000/1001,25,...,0.0,11,9,9,9,5,3,0,0,0


## Test & Train Split

In [12]:
df_Y = df_merged[['psnr']]
df_X = df_merged.drop('psnr', axis=1)
df_train, df_test, df_train_y, df_test_y = train_test_split(df_X, df_Y, test_size=0.2, random_state=0) 
print(df_train.shape, df_test.shape)
print(df_train_y.shape, df_test_y.shape)

(1735, 159) (434, 159)
(1735, 1) (434, 1)


## Label Binarizer: Converting set of categorical columns to numeric

In [13]:
categorical_columns =['encode_width', 'encode_height', 'clip_width', 'clip_height','video_profile', 'clip_frame_rate'] 

for column in categorical_columns:
    lb = preprocessing.LabelBinarizer()
    lb.fit(df_train[column].values.reshape(-1,1))
    dump(lb, f"{column}_label_binarizer.pkl")
    binarized_array_train = lb.transform(df_train[column].values.reshape(-1,1))
    column_names = [f"{column}_{i}" for i in list(lb.classes_)]
    
    binarized_df_train = pd.DataFrame(data=binarized_array_train, columns=column_names, index=df_train.index)
    df_train = pd.concat([df_train, binarized_df_train], axis=1, sort=False)
    df_train.drop(columns=[column], inplace=True)

    binarized_array_test = lb.transform(df_test[column].values.reshape(-1,1))
    binarized_df_test = pd.DataFrame(data=binarized_array_test, columns=column_names, index=df_test.index)
    df_test = pd.concat([df_test, binarized_df_test], axis=1, sort=False)
    df_test.drop(columns=[column], inplace=True)
    print(column + ' is converted')
    print(df_train.shape, df_test.shape)

encode_width is converted
(1735, 165) (434, 165)
encode_height is converted
(1735, 170) (434, 170)
clip_width is converted
(1735, 170) (434, 170)
clip_height is converted
(1735, 172) (434, 172)
video_profile is converted
(1735, 174) (434, 174)
clip_frame_rate is converted
(1735, 177) (434, 177)


# Normalization of numerical variables

In [14]:
numerical_columns =['clip_duration', 'clip_size', 'clip_bitrate_total', 'encode_bitrate_video','crf'] 

for column in numerical_columns:
    normalizer = MinMaxScaler()
    normalizer.fit(df_train[column].values.reshape(-1,1))
    dump(normalizer, f"{column}_normalizer.pkl")
    df_train[column] = normalizer.transform(df_train[column].values.reshape(-1,1))
    df_test[column] = normalizer.transform(df_test[column].values.reshape(-1,1))

/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


## Export datasets: Test & Train

In [15]:
export_df_train = df_train.to_csv (r'export_df_train_psnr.csv', index = None, header=True) 

In [16]:
export_df_test = df_test.to_csv (r'export_df_test_psnr.csv', index = None, header=True) 

In [17]:
export_df_train_y = df_train_y.to_csv (r'export_df_train_y_psnr.csv', index = None, header=True) 

In [18]:
export_df_test_y = df_test_y.to_csv (r'export_df_test_y_psnr.csv', index = None, header=True) 

### / please use exported datasets to build models /